# ENEI-2022: Data wrangling

In [13]:
import os
import json
import pandas as pd
from typing import Tuple
from janitor import clean_names

## Import data

In [2]:
def load_data(path: str) -> Tuple[pd.DataFrame]:
    survey = pd.read_excel(os.path.join(path, "enei.xlsx"))
    features = pd.read_excel(os.path.join(path, "dict.xlsx"), skiprows=4).iloc[:318, :]
    factors = pd.read_excel(os.path.join(path, "dict.xlsx"), skiprows=326)
    return clean_names(survey), clean_names(features), clean_names(factors)

survey_raw, features_raw, factors_raw = load_data(path=os.path.join(os.getcwd(), "raw"))

## Filter surveys by the proportion of nans in columns

In [3]:
def remove_columns_with_nans(df, beta):
    nan_proportion = df.isna().mean()
    columns_to_drop = nan_proportion[nan_proportion >= beta].index
    df_filtered = df.drop(columns=columns_to_drop)
    return df_filtered

survey_filtered = remove_columns_with_nans(survey_raw, 0.2).dropna(axis=0)
survey_filtered

,dominio,areag,factor,upm,hogar_num,id,p03a02,p03a03,p03a05,p03a06,...,p07a04a,p07a05a,p07a06a,p07a07a,p07a08a,p07a09a,p07a10a,p07a11a,p07a12a,pet
0,1,1,386,100011,2721,1,2,75,1,3,...,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
1,1,1,386,100011,2721,2,1,35,3,3,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
2,1,1,386,100011,2721,3,2,35,4,3,...,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
3,1,1,386,100011,2721,4,2,15,5,3,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
4,1,1,386,100011,5021,1,2,75,1,3,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20168,3,2,1353,725722,2254,1,1,63,1,3,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
20169,3,2,1353,725722,2254,2,2,63,2,3,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
20170,3,2,1353,725722,2269,1,1,56,1,3,...,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
20171,3,2,1353,725722,2269,2,2,46,2,3,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0


## Take a look: what features are we dropping?

In [10]:
features = (
    features_raw
    .assign(
        nombre_de_la_variable = lambda x: x.nombre_de_la_variable.str.lower(),
        etiqueta_de_la_variable = lambda x: x.etiqueta_de_la_variable.str.lower()
    )
    .drop('posicion', axis=1)
)

features_description = dict(zip(features['nombre_de_la_variable'], features['etiqueta_de_la_variable']))

In [21]:
filtered_features_description = {key: value for key, value in features_description.items() if key in list(survey_filtered.columns)}
dropped_features_description = {key: value for key, value in features_description.items() if key not in list(survey_filtered.columns)}

names_and_dicts = {
    'filtered_features_description': filtered_features_description,
    'dropped_features_description': dropped_features_description
}

path = os.path.join(os.getcwd(), "features")
os.makedirs(path, exist_ok=True)

for name, dictionary in names_and_dicts.items():
    with open(os.path.join(os.getcwd(), "features", f"{name}.json"), "w") as file:
        json.dump(dictionary, file, ensure_ascii=False, indent=4)